### SEARCH ENGINE WITH TOOLS AND AGENTS 


In [ ]:
## Arxiv -- Research
## Tools creation
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun

from langchain_community.utilities.arxiv import ArxivAPIWrapper
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper


In [ ]:
## Used the inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name


In [ ]:
# --- Correct Arxiv Setup ---
api_wrapper_arxiv = ArxivAPIWrapper(
    top_k_results=1, 
    doc_content_chars_max=250
)
# Pass the instantiated wrapper to the tool's REQUIRED 'api_wrapper' field
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv, name="arxiv")

In [ ]:
tools = [wrapped_wiki,arxiv]


In [ ]:
## Custom tool [RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os 
from dotenv import load_dotenv
load_dotenv()
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs =  loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,embeddings)
retriever = vectordb.as_retriever()
retriever

In [ ]:
from langchain_core.tools import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langsmith-search", "Search any information about Langsmith")
retriever_tool.name

In [ ]:
tools = [wrapped_wiki,arxiv,retriever_tool]
tools

In [ ]:
##Run all the tools with Agents and LLM Models
## Tools,LLM-> Agent Executor
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key=groq_api_key, model_name="llama-3.3-70b-versatile")


In [ ]:
# Simplified System Prompt
system_prompt = """You are a helpful assistant. You have access to tools to answer questions. Use them when necessary.

When using the wikipedia tool, provide the input as a dictionary with a 'query' key. For example: {'query': 'Capital of France'}.You have access to the following tools:

- wikipedia: Use this tool to find general information and answer questions about various topics.
- arxiv: Use this tool to find research papers and information from scientific publications.
- langsmith-search: Use this tool to find information about Langsmith.

When answering questions, use the tools if needed to gather information. If you don't know the answer, say that you don't know."""

In [ ]:
###
def get_system_message_from_prompt(prompt):
    """
    Returns a string system message if found, otherwise None.
    Handles tuple shorthand, message objects, and PromptTemplate wrappers.
    """
    for msg in prompt.messages:
        # tuple/list shorthand: ("system", "text")
        if isinstance(msg, (list, tuple)) and len(msg) >= 2:
            if msg[0] == "system":
                return msg[1]

        # Real message objects (SystemMessage, HumanMessage, AIMessage)
        # these usually expose .content and sometimes .role
        if hasattr(msg, "content"):
            role = getattr(msg, "role", None) or msg.__class__.__name__
            # class name might be "SystemMessage"
            if role == "SystemMessage" or (isinstance(role, str) and role.lower() == "system"):
                return msg.content

        # Prompt-template objects like SystemMessagePromptTemplate -> .prompt.template
        tpl = getattr(getattr(msg, "prompt", None), "template", None)
        if tpl:
            # prefer explicit SystemMessagePromptTemplate class name or role attr
            if msg.__class__.__name__ == "SystemMessagePromptTemplate" or getattr(msg, "role", None) == "system":
                return tpl

    return None
###


In [ ]:
from langchain.agents import create_agent

# Create agent
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_prompt,
)

# Invoke the agent
query = "tell me about langsmith."
result = agent.invoke({"messages": [{"role": "user", "content": query}]})
print(result["messages"][-1].content)